In [6]:
import pandas as pd
import sqlite3
import json
import urllib.request

In [7]:
# Copied from anki connect site
def request(action: str, **params) -> dict:
    return {"action": action, "params": params, "version": 6}


# Copied from anki connect site
def invoke(action: str, **params):
    requestJson = json.dumps(request(action, **params)).encode("utf-8")
    response = json.load(
        urllib.request.urlopen(
            urllib.request.Request("http://127.0.0.1:8765", requestJson)
        )
    )
    if len(response) != 2:
        raise Exception("response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("response is missing required error field")
    if "result" not in response:
        raise Exception("response is missing required result field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]

In [4]:
db_path: str = "data/clean/bir_warehouse.db"

year_num_options: list = [[4, 2015], [5, 2014]]

query_question_text: str = """
SELECT qn.question, ex.exam_type, yr.year_name, qo.option_text, qo.is_correct
FROM questions_options AS qo
JOIN questions AS qn ON qo.question_id = qn.id
JOIN year AS yr ON qn.exam_year = yr.id_year
JOIN exam AS ex ON qn.exam_subject = ex.id_type;
"""

conn = sqlite3.connect(db_path)

exams_df = pd.read_sql(query_question_text, conn)

conn.close()
exams_df.head()

,question,exam_type,year_name,option_text,is_correct
0,1. La barrera hematoencefálica:,bir,2024,1. Es permeable a todas las sustancias present...,0
1,1. La barrera hematoencefálica:,bir,2024,2. Es permeable al O2 y al CO2.,1
2,1. La barrera hematoencefálica:,bir,2024,3. Es impermeable al etanol.,0
3,1. La barrera hematoencefálica:,bir,2024,4. Es impermeable al agua.,0
4,2. El espacio subaracnoideo se encuentra:,bir,2024,1. Entre la aracnoides y la duramadre.,0


In [5]:
print(len(exams_df))

24286
